In [ ]:
options(repr.plot.width = 12, repr.plot.height = 6)

# EX 8.7 모의 실험 데이터

In [ ]:
z <- scan("DAT/eg8_7.txt")
forecast::tsdisplay(z)

In [ ]:
forecast::ggtsdisplay(z,
        smooth=T)

In [ ]:
fit <- arima(z, order=c(1,0,0), method='ML')
fit

In [ ]:
mean(z)

$\hat Z_n(l) = \hat μ + \hat ϕ^l(Z_n − \hat μ)$

In [ ]:
forecast_fit <- forecast::forecast(fit, 25) #MMSE : zn(l) = mu + phi^l*(zn-mu), l=1,...,10
forecast_fit

In [ ]:
coef(fit)[2]

In [ ]:
hat_phi <- coef(fit)[1]
hat_mu <- coef(fit)[2]

In [ ]:
hat_mu + hat_phi * (z[100] - hat_mu) # l= 1 z_100_(1)
hat_mu + hat_phi^2 * (z[100] - hat_mu) # l= 2 z_100_(2)

In [ ]:
l <- 1:10
sapply(l, function(x) hat_mu + hat_phi^x * (z[100] - hat_mu))

In [ ]:
plot(forecast_fit)

In [ ]:
ggplot2::autoplot(forecast_fit)

In [ ]:
sarima_fit <- astsa::sarima.for(z,25,1,0,0)

In [ ]:
sarima_fit$pred
sarima_fit$se

In [ ]:
head(sarima_fit$pred + qnorm(0.975)*sarima_fit$se) ##95% 신뢰구간 상한
head(sarima_fit$pred - qnorm(0.975)*sarima_fit$se) ##95% 신뢰구간 하한

# EX 9.5 IMA(1,1) = ARIMA(0,1,1)

In [ ]:
z <- scan("DAT/eg9_5.txt")
forecast::ggtsdisplay(z, smooth=T)

In [ ]:
##단위근 검정 : H0 : 단위근이 있다.
fUnitRoots::adfTest(z, lags = 0, type = "c")
fUnitRoots::adfTest(z, lags = 1, type = "c")
fUnitRoots::adfTest(z, lags = 2, type = "c")

In [ ]:
lag_z <- diff(z)
forecast::tsdisplay(lag_z)

In [ ]:
## mean : H0 : mu = 0
t.test(lag_z)

In [ ]:
fit1 <- arima(lag_z, order=c(0,0,1), include.mean = F)
fit1

In [ ]:
fit <- arima(z, order=c(0,1,1))
fit

$Z_t = ε_t + 0.7605ε_t, \hat θ = −0.7605$

In [ ]:
forecast::checkresiduals(fit)

In [ ]:
resid = resid(fit)
forecast::tsdisplay(resid)

In [ ]:
# 잔차의 포트맨토 검정 ## H0 : rho1=...=rho_k=0
#portes::LjungBox(fit, lags=c(6,12,18,24))

In [ ]:
## 정규성검정
tseries::jarque.bera.test(resid) ##JB test H0: normal

In [ ]:
par(mfrow=c(1,2))
hist(resid)

qqnorm(resid, pch=16)
qqline(resid)

In [ ]:
## 잔차 검정
astsa::sarima(z, p=0, d=1, q=1)

In [ ]:
fore_fit <- forecast::forecast(fit, 25)
fore_fit

In [ ]:
plot(fore_fit)

In [ ]:
astsa::sarima.for(z, 25, 0,1,1)

In [ ]:
astsa::sarima.for(lag_z, 25, 0,0,1)

In [ ]:
fore_diff_z <- astsa::sarima.for(lag_z, 25, 0,0,1, plot=F)$pred
fore_diff_z

In [ ]:
fore_z <- astsa::sarima.for(z, 25, 0,1,1, plot=F)$pred
fore_z

$\widehat{▽Z_t} = \hat Z_t − \hat Z_{t−1}$

$\hat Z_t = \widehat{▽Z_t} + \hat Z_{t−1}$

$\hat Z_{300}(1) = \widehat{▽Z_{300}(1)} + Z_300$

In [ ]:
z_300_1 <- as.numeric(fore_diff_z[1]) + z[300]
z_300_1

$\hat Z_{300}(2) = \widehat{▽Z_{300}(2)} + \hat Z_{300}(1)$

In [ ]:
z_300_2 <- fore_diff_z[2] + z_300_1
z_300_2